In [35]:
import numpy as np

In [36]:
S_COUNT = 81
A_COUNT = 4
GAMMA = 0.9925

In [37]:
def parse(file_name):
    # s, s', p(s'|s,a)
    f = np.loadtxt(file_name)
    trans_mtx = np.zeros([S_COUNT, S_COUNT])
    for line in f:
        s, s_, p = int(line[0])-1, int(line[1])-1, float(line[2])
        trans_mtx[s, s_] = p
    return trans_mtx

In [38]:
prob_a1 = parse('prob_a1.txt')
prob_a2 = parse('prob_a2.txt')
prob_a3 = parse('prob_a3.txt')
prob_a4 = parse('prob_a4.txt')
rewards = np.loadtxt('rewards.txt')

## (a) Policy Iteration

In [39]:
trans_matrices = [prob_a1, prob_a2, prob_a3, prob_a4]

In [40]:
# init policy at random
policy = np.random.randint(low=0,high=A_COUNT, size=S_COUNT)
policy.shape

(81,)

In [41]:
def policy_eval(policy, trans_matrices, rewards):
    # construct p_pi
    p_pi = np.zeros((S_COUNT, S_COUNT))
    for i in range(S_COUNT):
        p_pi[i] = trans_matrices[policy[i]][i]
    v_pi = np.linalg.inv(np.identity(S_COUNT) - GAMMA * p_pi).dot(rewards)
    return v_pi

In [42]:
def policy_improvement(policy, trans_matrices, v_pi):
    new_policy = np.copy(policy)
    for s in range(S_COUNT):
        tmp = np.zeros(A_COUNT)
        for s_ in range(S_COUNT):
            for i in range(A_COUNT):
                tmp[i] += trans_matrices[i][s, s_] * v_pi[s_]
        new_policy[s] = np.argmax(tmp)
    return new_policy

In [43]:
v_pi = np.ones(S_COUNT)
v_pi_prev = np.zeros(S_COUNT)
counter = 0
while max(abs(v_pi - v_pi_prev)) > 0.000001:
    v_pi_prev = np.copy(v_pi)
    v_pi = policy_eval(policy, trans_matrices, rewards)
    policy = policy_improvement(policy, trans_matrices, v_pi)
    counter += 1

In [46]:
from tabulate import tabulate
v_pi_out = np.copy(v_pi)
v_pi_out.resize((9, 9))
v_pi_out = np.transpose(v_pi_out)
print(tabulate(v_pi_out))

-------  -------  -------  -------  -------  ---------  --------  ---------  -------
  0        0        0        0        0         0         0          0         0
  0      102.375  103.235  104.101    0      -133.333    81.3995  -133.333     0
100.701  101.524    0      104.975  103.781    90.9854   93.6717    81.3995    0
  0        0      106.778  105.889    0      -133.333    95.1729  -133.333     0
  0        0      107.675    0        0         0       108.343      0         0
  0      109.49   108.578    0        0      -133.333   109.584   -133.333     0
  0      110.409    0      114.163  115.122   116.088   123.643    125.25    133.333
  0      111.336  112.27   113.213    0       122.025   123.182    124.207     0
  0        0        0        0        0         0         0          0         0
-------  -------  -------  -------  -------  ---------  --------  ---------  -------


In [52]:
policy_out = np.empty(S_COUNT, dtype=str)
actions = ['←','↑','→','↓']
for i in range(S_COUNT):
    if v_pi[i] == 0:
        policy_out[i] = '#'
    elif v_pi[i] < 0:
        policy_out[i] = 'X'
    else:
        policy_out[i] = actions[policy[i]]

policy_out.resize((9, 9))
policy_out = np.transpose(policy_out)
print(tabulate(policy_out))#%%

-  -  -  -  -  -  -  -  -
#  #  #  #  #  #  #  #  #
#  →  →  ↓  #  X  ↓  X  #
→  ↑  #  ↓  ←  ←  ↓  ←  #
#  #  ↓  ←  #  X  ↓  X  #
#  #  ↓  #  #  #  ↓  #  #
#  ↓  ←  #  #  X  ↓  X  #
#  ↓  #  →  →  →  →  →  ←
#  →  →  ↑  #  →  →  ↑  #
#  #  #  #  #  #  #  #  #
-  -  -  -  -  -  -  -  -


## (b) Value Iteration

In [58]:
# init policy at random
policy = np.random.randint(low=0,high=A_COUNT, size=S_COUNT)
v_pi = np.ones(S_COUNT)
v_pi_prev = np.zeros(S_COUNT)
while max(abs(v_pi - v_pi_prev)) > 0.000001:
    for s in range(S_COUNT):
        tmp = np.zeros(A_COUNT)
        for s_ in range(S_COUNT):
            for i in range(A_COUNT):
                tmp[i] += trans_matrices[i][s, s_] * v_pi_prev[s_]
        v_pi_prev[s] = v_pi[s]
        v_pi[s] = rewards[s] + GAMMA * max(tmp)
        policy[s] = np.argmax(tmp)

In [59]:
v_pi_out = np.copy(v_pi)
v_pi_out.resize((9, 9))
v_pi_out = np.transpose(v_pi_out)
print(tabulate(v_pi_out))

-------------  -------------  -------------  -------------  -------------  --------------  -------------  --------------  -------------
  9.93634e-07    9.93634e-07    9.93634e-07    9.93634e-07    9.93634e-07     9.93634e-07    9.93634e-07     9.93634e-07    9.93634e-07
  9.93634e-07  102.375        103.235        104.101          9.93634e-07  -133.333         81.3994       -133.333          9.93634e-07
100.701        101.524          9.93634e-07  104.975        103.781          90.9853        93.6716         81.3994         9.93634e-07
  9.93634e-07    9.93634e-07  106.778        105.888          9.93634e-07  -133.333         95.1728       -133.333          9.93634e-07
  9.93634e-07    9.93634e-07  107.675          9.93634e-07    9.93634e-07     9.93634e-07  108.343           9.93634e-07    9.93634e-07
  9.93634e-07  109.49         108.578          9.93634e-07    9.93634e-07  -133.333        109.584        -133.333          9.93634e-07
  9.93634e-07  110.409          9.93634e-07  114

In [60]:
policy_out = np.empty(S_COUNT, dtype=str)
actions = ['←','↑','→','↓']
for i in range(S_COUNT):
    if v_pi[i] < 0:
        policy_out[i] = 'X'
    elif v_pi[i] <= 0.000001:
        policy_out[i] = '#'
    else:
        policy_out[i] = actions[policy[i]]

policy_out.resize((9, 9))
policy_out = np.transpose(policy_out)
print(tabulate(policy_out))#%%

-  -  -  -  -  -  -  -  -
#  #  #  #  #  #  #  #  #
#  →  →  ↓  #  X  ↓  X  #
→  ↑  #  ↓  ←  ←  ↓  ←  #
#  #  ↓  ←  #  X  ↓  X  #
#  #  ↓  #  #  #  ↓  #  #
#  ↓  ←  #  #  X  ↓  X  #
#  ↓  #  →  →  →  →  →  ←
#  →  →  ↑  #  →  →  ↑  #
#  #  #  #  #  #  #  #  #
-  -  -  -  -  -  -  -  -
